In [1]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7ff697c66510>
2.0.0+cu117


'%.4f'

In [2]:
lib.utils.set_seed()
downscale = (4, 4)
test_x, test_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_test(), downscale=downscale)
train_x, train_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_train(), downscale=downscale)

In [3]:
network = lib.livenet.LiveNet.create_perceptron(train_x.shape[1], 21, 10)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=100)


LiveNet


In [10]:
network.context.alpha_l1 = 0.001
trainer.step(500)
# network.context.alpha_l1 = 0.01
# trainer.step(5000)



Iˈ0.000 2.649 = 2.380+0.268 #nparams=905 lib/trainer.py:93
Iˈ3.869 N1 became dangle, total dangle = 1 lib/death.py:55
Iˈ10.610 S64 became useless at tick 1079 lib/livenet.py:61
Iˈ12.620 S13 became useless at tick 1093 lib/livenet.py:61
Iˈ13.855 2.635 = 2.385+0.251 #nparams=793 lib/trainer.py:93
Iˈ17.671 N1 became useless at tick 1128 lib/livenet.py:61
Iˈ17.672 *********** <class 'life.lib.livenet.RegularNeuron'> lib/livenet.py:153
Iˈ17.672 *********** N1 lib/livenet.py:154


TypeError: super(type, obj): obj must be an instance or subtype of type

In [8]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.5901

In [9]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [7]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")

D0 -0.7
D1 -0.3
D2 1.9
D3 1.8
D4 1.9
D5 2.0
D6 2.3
D7 1.3
D8 -1.6
D9 -1.1
N0 0.9
N12 1.6
N19 1.2
N0->D7 0.5
N0->D9 1.1
N12->D0 1.6
N12->D6 -0.6
N12->D8 1.6
N19->D1 1.6
N19->D8 0.6
N19->D9 1.2
S129->N0 1.2
S20->N12 -0.6
S63->N12 -1.0
S71->N19 0.3
S129->N12 0.6
S134->N12 0.3
S147->N12 0.2
S148->N12 0.7
S154->N12 0.4
S157->N12 0.3
S172->N12 0.2
S183->N12 0.6
S105->N19 -0.6
S110->N19 -0.5
S114->N19 -0.9
S116->N19 -0.7
S131->N19 0.1
S156->N19 0.6
S157->N19 0.0
S163->N19 0.4
S184->N19 0.6


In [19]:
network._parameters

OrderedDict([('D0',
              Parameter containing:
              tensor(-0.0146, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(0.0146, requires_grad=True)),
             ('N4',
              Parameter containing:
              tensor(-0.0045, requires_grad=True))])

In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

